In [53]:
import random
import math

from tqdm import tqdm

In [54]:
def propagate_outer():
    pass

class MyMLP:

    def __init__(self, neurons_per_layer: list[int]):
        self.d = list(neurons_per_layer)  # neurons_per_layer (input included)
        self.L = len(self.d) - 1
        self.W = []
        #self.W[l][i][j]
        for l in range(len(self.d)):
            self.W.append([])
            if l == 0:
                continue
            for i in range(self.d[l - 1] + 1):  # +1 compte le neurone de biais fictif
                self.W[l].append([])
                for j in range(self.d[l] + 1):
                    rdm = random.random() * 2 - 1
                    self.W[l][i].append(0.0 if j == 0 else rdm)
        self.X = []
        self.deltas = []

        for l in range(len(self.d)):
            self.X.append([])
            self.deltas.append([])

            for j in range(self.d[l] + 1):
                self.X[l].append(1.0 if j == 0 else 0.0)
                self.deltas[l].append(0.0)

    def _propagate(self, inputs: list[float], is_classification : bool):
        assert len(inputs) == self.d[0]
        for j in range(len(inputs)):
            self.X[0][j+1] = inputs[j]

        for l in range(1, self.L+1):
            for j in range(1, self.d[l] + 1):
                signal = 0
                for i in range(self.d[l-1] + 1):
                    signal += self.W[l][i][j] * self.X[l - 1][i]
                x = signal
                if is_classification or l != self.L:
                    x = math.tanh(signal)
                self.X[l][j] = x

    def predict(self, inputs: list[float], is_classification: bool):
        self._propagate(inputs, is_classification)
        return self.X[self.L]
    
    def train(self, 
              all_samples_inputs:list[list[float]],
                all_samples_expected_outputs: list[list[float]],
                is_classification: bool,
                num_iter: int,
                alpha: float # learning rate
              
              ):
        assert len(all_samples_inputs) == len(all_samples_expected_outputs)

        for _ in tqdm(range(num_iter)):
            k = random.randint(0, len(all_samples_inputs) - 1)
            inputs_k = all_samples_inputs[k]
            outputs_k = all_samples_expected_outputs[k]

            self._propagate(inputs_k, is_classification)  # mise a jour des xij
            for j in range(self.d[self.L] + 1):
                # self.deltas[self.L][j] = (1.0 - self.X[self.L][j] ** 2) * (self.X[L])
                if is_classification:
                    self.deltas[self.L][j] *= (1.0 - self.X[self.L][j] ** 2)

            for l in reversed(range(2, self.L + 1)):
                for i in range(1, self.d[self.L-1] + 1):
                    total = 0
                    for j in range(1, self.d[l] + 1):
                        total += self.W[l][i][j] * self.deltas[i][j]
                    total *= (1.0 - self.X[l - 1][i] ** 2)
                    self.deltas[l - 1][i] = total
            for l in range(1, self.L + 1):
                for i in range(self.d[i-1] + 1):
                    for j in range(1, self.d[l] + 1):
                        self.W[l][i][j] = self.W[l][i][j] - alpha * self.X[l-1][i] * self.deltas[l][j]

In [55]:
mlp = MyMLP([2, 3, 1])
mlp.W
print(mlp.predict([-42.0, 42.0], True))

[1.0, 0.7393354049350186]


In [56]:
xor_inputs = [
    [0., 0.],
    [1., 0.],
    [0., 1.],
    [1., 1.]
]

xor_outputs = [
    [-1.],
    [1.],
    [1.],
    [-1.0]
]

for sample in xor_inputs:
    print(mlp.predict(sample, True))

mlp.train(xor_inputs, xor_outputs, True, 10000, 0.01)

for sample in xor_inputs:
    print(mlp.predict(sample, True))


[1.0, 0.08614693298269892]
[1.0, 0.19847343792666028]
[1.0, 0.4320379896678563]
[1.0, 0.5031926585772118]


  0%|          | 0/10000 [00:00<?, ?it/s]


IndexError: list index out of range